## M1 - Linear State Class v1

In [ ]:
# Model Run 01
# Clean & convert types
reg_df["tot_appr_val"] = pd.to_numeric(reg_df["tot_appr_val"], errors="coerce")
reg_df["state_class_y_desc"] = reg_df["state_class_y_desc"].astype(str)
reg_df = reg_df.dropna(subset=["tot_appr_val", "state_class_y_desc"])

# Create dummy vars and add constant
X = pd.get_dummies(reg_df["state_class_y_desc"], drop_first=True)
X = sm.add_constant(X)

y = reg_df["tot_appr_val"]

# Convert to NumPy float arrays
X_np = X.astype(float).values
y_np = y.astype(float).values

# Fit model
model = sm.OLS(y_np, X_np).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     86.38
Date:                Sun, 18 May 2025   Prob (F-statistic):               0.00
Time:                        14:20:24   Log-Likelihood:            -2.7781e+05
No. Observations:               17200   AIC:                         5.557e+05
Df Residuals:                   17173   BIC:                         5.559e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.171e-08   1.77e+06  -3.48e-14      1.0

Observations: 

Dependent Variable: tot_appr_val

Only ~5.1% of the variation in appraised value is explained by property class alone.

The model is statistically significant overall.

R² is low → Property class alone is not a strong predictor of value

Jarque-Bera test and skew = 7.3 → distribution is heavily non-normal

## M2 - Linear State Class v2

In [ ]:
# Model Run 02

# Step 1: Copy and clean
reg_df = df_corr.copy()

# Step 2: Convert relevant columns
reg_df["tot_appr_val"] = pd.to_numeric(reg_df["tot_appr_val"], errors="coerce")
reg_df["yr_impr"] = pd.to_numeric(reg_df["yr_impr"], errors="coerce")
reg_df["acreage"] = pd.to_numeric(reg_df["acreage"], errors="coerce")
reg_df["state_class_y_desc"] = reg_df["state_class_y_desc"].astype(str)

# Step 3: Drop rows with missing values
reg_df = reg_df.dropna(subset=["tot_appr_val", "yr_impr", "acreage", "state_class_y_desc"])

# Step 4: Log-transform value
reg_df["log_val"] = np.log1p(reg_df["tot_appr_val"])

# Step 5: Create design matrix
X = pd.get_dummies(reg_df[["state_class_y_desc"]], drop_first=True)
X["yr_impr"] = reg_df["yr_impr"]
X["acreage"] = reg_df["acreage"]

# Step 6: Add constant and coerce to float
X = sm.add_constant(X)
X = X.astype(float)
y = reg_df["log_val"].astype(float)

# Step 7: Fit the model using NumPy arrays
model = sm.OLS(y.values, X.values).fit()

# Step 8: Output summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.307
Method:                 Least Squares   F-statistic:                     273.1
Date:                Sun, 18 May 2025   Prob (F-statistic):               0.00
Time:                        14:20:25   Log-Likelihood:                -37077.
No. Observations:               17200   AIC:                         7.421e+04
Df Residuals:                   17171   BIC:                         7.444e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0273      2.006     -1.509      0.1

Observations 

Model now explains 28.1% of the variance in assessed values.

Each year older decreases log value by ~1% (statistically significant, p = 0.019)

Each additional acre increases log value by ~24% (strongest continuous predictor)



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

### M2 Graphs

In [ ]:
### Old Charts
# Step 1: Predict log values
reg_df["log_val_pred"] = model.predict(X.values)

# Step 2: Plot fitted vs actual
sns.jointplot(
    x=reg_df["log_val_pred"],
    y=reg_df["log_val"],
    kind="reg",
    height=6,
    scatter_kws={"alpha": 0.5, "edgecolor": "k", "linewidths": 0.3}
).set_axis_labels("Predicted Log Value", "Actual Log Value")

plt.suptitle("Fitted vs Actual (Log Appraised Value)", y=1.02, fontsize=14)
plt.show()
# Step 1: Predicted values and residuals
reg_df["log_val_pred"] = model.predict(X.values)
reg_df["residuals"] = reg_df["log_val"] - reg_df["log_val_pred"]

# Step 2: Plot
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=reg_df,
    x="log_val_pred",
    y="residuals",
    alpha=0.5,
    edgecolor="k",
    linewidths=0.3
)
plt.axhline(0, color="red", linestyle="--", linewidth=1)
plt.title("Residuals vs Fitted Values (Log Scale)")
plt.xlabel("Predicted Log Appraised Value")
plt.ylabel("Residual (Actual − Predicted)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(
    data=reg_df,
    x="state_class_y_desc",
    y="residuals"
)
plt.axhline(0, color="red", linestyle="--", linewidth=1)
plt.title("Residuals by Property Class")
plt.xlabel("State Class")
plt.ylabel("Residual (Log Actual − Log Predicted)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Create residuals column
reg_df["log_val_pred"] = model.predict(X.values)
reg_df["residuals"] = reg_df["log_val"] - reg_df["log_val_pred"]

# Grouping market area into categories
reg_df["market_group"] = "Other"
reg_df.loc[reg_df["Market_Area_1_Dscr"].str.startswith("ISD", na=False), "market_group"] = "ISD"
reg_df.loc[reg_df["Market_Area_1_Dscr"].str.match(r"^[A-Z0-9]{2}\\s", na=False), "market_group"] = "Code Prefix"

# Boxplot by grouped market area
g = sns.boxplot(
    data=reg_df,
    x="market_group",
    y="residuals"
)
g.axhline(0, color="red", linestyle="--", linewidth=1)
g.set_title("Residuals by Market Area Group")
g.set_xlabel("Market Area Group")
g.set_ylabel("Residual (Log Actual − Log Predicted)")
plt.xticks(rotation=90, ha="right")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(
    data=reg_df[reg_df["market_group"] == "ISD"],
    x="Market_Area_1_Dscr",
    y="residuals"
)
plt.axhline(0, color="red", linestyle="--", linewidth=1)
plt.title("Residuals by ISD Market Areas")
plt.xlabel("ISD Market Area")
plt.ylabel("Residual (Log Actual − Log Predicted)")
plt.xticks(rotation=90, ha="right")
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(
    data=reg_df[reg_df["market_group"] != "ISD"],
    x="Market_Area_1_Dscr",
    y="residuals"
)
plt.axhline(0, color="red", linestyle="--", linewidth=1)
plt.title("Residuals by Non-ISD Market Areas")
plt.xlabel("Market Area")
plt.ylabel("Residual (Log Actual − Log Predicted)")
plt.xticks(rotation=90, ha="right")
plt.tight_layout()
plt.show()

# Filter for strong underprediction (log residual <= -5)
underpred_df = reg_df[reg_df["residuals"] <= -5].copy()

# Sort by most extreme underpredictions
underpred_df = underpred_df.sort_values(by="residuals")

# Select columns of interest
cols = [
    "tot_appr_val", "log_val", "log_val_pred", "residuals",
    "state_class_y_desc", "Market_Area_1_Dscr", "acreage", "yr_impr"
]
underpred_table = underpred_df[cols]
underpred_table
# Filter underpredicted rows
underpred_df = reg_df[reg_df["residuals"] <= -5].copy()

grouped_summary = (
    underpred_df
    .groupby(["Market_Area_1_Dscr", "state_class_y_desc"])
    .agg(
         parcel_count=pd.NamedAgg(column="residuals", aggfunc="count"),
        avg_residual=("residuals", "mean"),
        min_residual=("residuals", "min"),
        max_residual=("residuals", "max"),
        median_val=("tot_appr_val", "median"),
        avg_val=("tot_appr_val", "mean")
    )
    .sort_values(by=["parcel_count"], ascending=False)
    .reset_index()
)
display(grouped_summary.round(2))
Observation

While the residuals have abnormal residuals, it is still few in count compared to the entire service area. 